To show how to make a set of photons from a 3D dataset using pyXSIM and yt for reading into SOXS, we'll look at is that of thermal emission from a galaxy cluster. In this case, the gas in the core of the cluster is "sloshing" in the center, producing spiral-shaped cold fronts. The dataset we want to use for this example is available for download from the [yt Project](http://yt-project.org) at [this link](http://yt-project.org/data/GasSloshing.tar.gz). 

First, import our necessary modules:

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rc("font", size=18)
import yt
import pyxsim
import soxs

Next, we `load` the dataset with yt:

In [ ]:
ds = yt.load("GasSloshing/sloshing_nomag2_hdf5_plt_cnt_0150")

Let's use yt to take a slice of density and temperature through the center of the dataset so we can see what we're looking at: 

In [ ]:
slc = yt.SlicePlot(ds, "z", ["density", "temperature"], width=(1.0,"Mpc"))
slc.show()

Ok, sloshing gas as advertised. Next, we'll create a sphere object to serve as a source for the photons. Place it at the center of the domain with `"c"`, and use a radius of 500 kpc:

In [ ]:
sp = ds.sphere("c", (1.0, "Mpc"))

Now, we need to set up a spectral model. We said we were going to look at the thermal emission from the hot plasma, so to do that we can set up a thermal spectrum. We'll do that here by creating an `TableApecModel`. The first three arguments are the maximum and minimum energies, and the number of bins in the spectrum. We've chosen these numbers so that the spectrum has an energy resolution of about 1 eV. Setting `thermal_broad=True` turns on thermal broadening. 

In [ ]:
spec_model = pyxsim.TableApecModel(0.05, 11.0, 10000, thermal_broad=True)

Now that we've created a spectral model, we need to use it for our source model, which is that of a thermal source, buy using `ThermalSourceModel`. It takes a lot of different arguments, which you can investigate in the docs, but here we'll do something simple and say that it uses the above spectral model and the metallicity is a constant $Z = 0.3~Z_\odot$:

In [ ]:
source_model = pyxsim.ThermalSourceModel(spec_model, Zmet=0.3)

We're almost ready to go to generate the photons from this source, but first we should decide what our redshift, collecting area, and exposure time should be. Let's pick big numbers, because remember the point of this first step is to create a Monte-Carlo sample from which to draw smaller sub-samples for mock observations. Note these are all (value, unit) tuples:

In [ ]:
exp_time = (500., "ks") # exposure time
area = (30000.0, "cm**2") # collecting area
redshift = 0.2

So, that's everything--let's create the photons!

In [ ]:
photons = pyxsim.PhotonList.from_data_source(sp, redshift, area, exp_time, source_model)

Ok, that was easy. Now we have a `PhotonList` that we can use to create events. To be realistic, we're going to want to create an foreground Galactic absorption model, using the "TBabs" absorption model and assuming a foreground absorption column of $N_H = 4 \times 10^{20}~{\rm cm}^{-2}$:

In [ ]:
tbabs_model = pyxsim.TBabsModel(0.04)

Once we have created this, we can project the photons. Here we'll just do a simple projection along the z-axis, reducing the exposure time, and centering the photons at RA, Dec = (30, 45) degrees:

In [ ]:
events_z = photons.project_photons("z", absorb_model=tbabs_model, sky_center=(30.,45.))

Now that we have a set of "events" on the sky, we can write them to a SIMPUT file:

In [ ]:
events_z.write_simput_file("sloshing", clobber=True)

We can then use this SIMPUT file as an input to the instrument simulator in SOXS. We'll use a small exposure time (100 ks instead of 500 ks), and observe it with the calorimeter:

In [ ]:
soxs.instrument_simulator("sloshing_simput.fits", "evt.fits", 100000.0, "mucal", [30., 45.], overwrite=True)

We can use the `write_image()` function in SOXS to bin the events into an image and write them to a file, restricting the energies between 0.5 and 2.0 keV:

In [ ]:
soxs.write_image("evt.fits", "img.fits", emin=0.5, emax=2.0, overwrite=True)

We can use [APLpy](https://aplpy.github.io/) to show the resulting image:

In [ ]:
import aplpy
fig = aplpy.FITSFigure("img.fits")
fig.show_colorscale(vmax=1000.0, stretch='sqrt', cmap="afmhot")
fig.recenter(30., 45., width=0.05, height=0.05) # Centered on the source center with a width of 3 arcmin

We can also bin the events into a spectrum using `write_spectrum()` and write the spectrum to disk:

In [ ]:
soxs.write_spectrum("evt.fits", "evt.pha", overwrite=True)

and plot the spectrum using `plot_spectrum()`:

In [ ]:
fig = soxs.plot_spectrum("evt.pha")
fig.axes[0].set_xscale("log")
fig.axes[0].set_yscale("log")
fig.axes[0].set_xlim(0.5, 7.0)

Let's zoom into the region of the spectrum around the iron line to look at the detailed structure afforded by the resolution of the calorimeter:

In [ ]:
fig.axes[0].set_xlim(5.4, 5.7)
fig.axes[0].set_ylim(2.0e-5, 5.0e-3)
fig